# TD learning; the agent that can see

# Remember to check the number of samples for alpha and beta

now I'm gonna add numbers to the model. 

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
np.random.seed(42)
from bayes_opt import BayesianOptimization
from joblib import Parallel, delayed



In [13]:
folder_path = 'data_risk_added'
dataframes = [pd.read_excel(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.xlsx')]

n_participant = len(dataframes)
print(f"there are {n_participant} participants.")


output_dir = "12_RL_agent_TDlearn_output"
os.makedirs(output_dir, exist_ok=True)
dataframes[0]


there are 31 participants.


,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,risk
0,570,uniform,831,lose,5,2,2209,9.5,0,response,arrowdown,1,0,0.500
1,1162,uniform,901,lose,4,3,5755,9,1,response,arrowdown,1,0,0.375
2,355,uniform,939,win,4,6,1209,9.5,2,response,arrowdown,1,0,0.375
3,1163,uniform,828,win,7,5,1997,10,3,response,arrowup,1,0,0.250
4,299,uniform,776,win,3,9,1324,10.5,4,response,arrowdown,1,0,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,667,uniform,997,lose,5,4,1290,71,43,response,arrowdown,4,0,0.500
267,202,low,944,lose,2,1,1111,70.5,72,response,arrowdown,4,0,0.243
268,297,high,833,win,9,6,1049,71,90,response,arrowup,4,0,0.000
269,1459,high,884,win,7,8,1132,71.5,98,response,arrowdown,4,0,0.447


# policy initilization for the model
now I need to find the prior policy amounts. for that I am going to put the percentage of downarrow and up arrow for each number 

In [14]:
df_combined = pd.concat(dataframes, ignore_index=True)

df_combined = df_combined[df_combined['outcome'].str.lower() != 'na']  



cards_sorted = sorted(df_combined["myCard"].unique())
choice_sorted = sorted(df_combined["choice"].unique())


card_idx = {card: i for i, card in enumerate(cards_sorted)}
choice_idx = {choice: i for i, choice in enumerate(choice_sorted)}


matrix_2d = np.zeros((len(cards_sorted), len(choice_sorted)))


for _, row in df_combined.iterrows():
    i = card_idx[row["myCard"]]-1
    k = choice_idx[row["choice"]]
    matrix_2d[i, k] += 1  


total_per_card_dist = matrix_2d.sum(axis=1, keepdims=True)

# compute percentages, avoiding division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    percentage_matrix = np.divide(matrix_2d, total_per_card_dist, where=total_per_card_dist != 0)

# convert to a DataFrame for easy visualization
percentage_list = []
for i, card in enumerate(cards_sorted):
    for k, choice in enumerate(choice_sorted):
        percentage_list.append({
            "myCard": card,
            "choice": choice,
            "percentage": percentage_matrix[i, k]
        })

df_percentages = pd.DataFrame(percentage_list)
df_percentages

,myCard,choice,percentage
0,1,arrowdown,0.943011
1,1,arrowup,0.056989
2,2,arrowdown,0.857595
3,2,arrowup,0.142405
4,3,arrowdown,0.709677
5,3,arrowup,0.290323
6,4,arrowdown,0.499466
7,4,arrowup,0.500534
8,5,arrowdown,0.337953
9,5,arrowup,0.662047


In [15]:
np.shape(percentage_matrix)
# percentage_matrix

(9, 2)

In [16]:
actions = { "arrowdown": 0, "arrowup": 1}
card_numbers = list(range(1, 10))

policy_table = percentage_matrix 

Q_table_init = np.random.normal(0, 0.1, (len(card_numbers), len(actions)))
# having a q-table based on the policies
Q_table_init = policy_table * np.mean(Q_table_init) 
Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))


policy: 
 (9, 2)

 Q_table: 
 (9, 2)


In [17]:
def softmax(Q_values, beta):    
    # this part subtracts the maximum q-value in each row it means each state to improve numerical stability.
    # because exxponentials of large numbers can lead to overflow errors, so shifting q-values avoids this problem.
    
    Q_shifted = Q_values - np.max(Q_values, axis=1, keepdims=True)
    exps = np.exp(beta * Q_shifted)
    sums = np.sum(exps, axis=1, keepdims=True)
    new_probs = exps / sums

    return new_probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["choice"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5
        
        probs = softmax(Q_values, beta)
        predicted_probs.append(probs[card_number][action])
        
        prediction_error = reward - Q_values[card_number][action]
        Q_values[card_number][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant)
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [18]:
BIC_models = []
for idx, df_all in enumerate(dataframes):
    Q_init_participant = Q_table.copy()
    
    df_all = df_all[df_all['outcome'].str.lower() != 'na']  

    num_of_samples = 50
    # num_of_samples = 1000
    alpha_samples = np.random.uniform(0.01, 1, num_of_samples) 
    beta_samples = np.random.uniform(0, 8, num_of_samples)  


    best_alpha, best_beta = None, None
    best_log_likelihood = -np.inf


    alpha_beta_log_likelihood = {}

    results = Parallel(n_jobs=-1)(delayed(compute_log_likelihood)(alpha, beta, df_all, Q_table) 
                                for alpha in alpha_samples for beta in beta_samples)


# finding alpha beta in parallel way
    alpha_beta_log_likelihood = {}
    best_log_likelihood = -np.inf
    best_alpha, best_beta = None, None

    for alpha, beta, log_likelihood in results:
        alpha_beta_log_likelihood[(alpha, beta)] = log_likelihood
        if log_likelihood > best_log_likelihood:
            best_log_likelihood = log_likelihood
            best_alpha, best_beta = alpha, beta


    results_df = pd.DataFrame(alpha_beta_log_likelihood.keys(), columns=["alpha", "beta"])
    results_df["log_likelihood"] = alpha_beta_log_likelihood.values()

    #  model prediction 
    
    q_values, choices, predicted_probs, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant)
    # now we need to find out the predicted choices of the model:
    

    predicted_choices = []

    for trial in range(len(card_numbers)):  
        if q_values[trial][card_numbers[trial]][actions["arrowup"]] > q_values[trial][card_numbers[trial]][actions["arrowdown"]]:
            predicted_choices.append(1)
        else:
            predicted_choices.append(0)

    
    
    # confusion matrix
    conf_matrix = confusion_matrix(choices, predicted_choices)
    
    # bayes information criterion
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood
    
    BIC_models.append(BIC)
    ###########################################################################################
    ## visulization

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))


    sns.kdeplot(x=results_df["alpha"], y=results_df["beta"], fill=True, cmap="viridis", ax=axes[0])
    mappable = axes[0].collections[0]
    fig.colorbar(mappable, ax=axes[0], label="density")
    axes[0].set_xlabel("learning rate (alpha)")
    axes[0].set_ylabel("inverse temp (beta)")
    axes[0].set_title("density of alpha beta joint probability")


    scatter = sns.scatterplot(x=results_df["alpha"], y=results_df["beta"], 
                            size=results_df["log_likelihood"], hue=results_df["log_likelihood"], 
                            palette="Blues", ax=axes[1], legend=False) 

    norm = plt.Normalize(results_df["log_likelihood"].min(), results_df["log_likelihood"].max())
    sm = plt.cm.ScalarMappable(cmap="Blues", norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=axes[1], label="log likelihood")

    axes[1].set_ylabel("inverse temp (beta)")
    axes[1].set_xlabel("learning rate (alpha)")
    axes[1].set_title("log likelihood scatterplot")




    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=["arrowdown", "arrowup"], 
                yticklabels=["arrowdown", "arrowup"], ax=axes[2], cbar=False)
    axes[2].set_xlabel("prediction")
    axes[2].set_ylabel("true label")
    axes[2].set_title(f"confusion matrix (α={best_alpha:.2f}, β={best_beta:.2f})")
    axes[2].text(0.5, -0.3, f"BIC: {BIC:.2f}", fontsize=12, fontweight="bold",
            ha="center", va="center", transform=axes[2].transAxes)
    


    plt.tight_layout(rect=[0, 0, 1, 0.9]) 
    fig.suptitle(f'participant {idx}', fontsize=16, fontweight='bold')

    filename = os.path.join(output_dir, f"plot_{idx}.pdf")
    plt.savefig(filename, format='pdf')
    plt.close(fig)

    print(f"saved: {filename}")


# saving BIC_models to compare models:
file_path_BIC = os.path.join(output_dir, "BIC_models_see_numbers.txt")

with open(file_path_BIC, "w") as file:
    for bic in BIC_models:
        file.write(f"{bic}\n")

print(f"BIC values saved to {file_path_BIC}")


saved: 12_RL_agent_TDlearn_output/plot_0.pdf
saved: 12_RL_agent_TDlearn_output/plot_1.pdf
saved: 12_RL_agent_TDlearn_output/plot_2.pdf
saved: 12_RL_agent_TDlearn_output/plot_3.pdf
saved: 12_RL_agent_TDlearn_output/plot_4.pdf
saved: 12_RL_agent_TDlearn_output/plot_5.pdf
saved: 12_RL_agent_TDlearn_output/plot_6.pdf
saved: 12_RL_agent_TDlearn_output/plot_7.pdf
saved: 12_RL_agent_TDlearn_output/plot_8.pdf
saved: 12_RL_agent_TDlearn_output/plot_9.pdf
saved: 12_RL_agent_TDlearn_output/plot_10.pdf
saved: 12_RL_agent_TDlearn_output/plot_11.pdf
saved: 12_RL_agent_TDlearn_output/plot_12.pdf
saved: 12_RL_agent_TDlearn_output/plot_13.pdf
saved: 12_RL_agent_TDlearn_output/plot_14.pdf
saved: 12_RL_agent_TDlearn_output/plot_15.pdf
saved: 12_RL_agent_TDlearn_output/plot_16.pdf
saved: 12_RL_agent_TDlearn_output/plot_17.pdf
saved: 12_RL_agent_TDlearn_output/plot_18.pdf
saved: 12_RL_agent_TDlearn_output/plot_19.pdf
saved: 12_RL_agent_TDlearn_output/plot_20.pdf
saved: 12_RL_agent_TDlearn_output/plot_21.pd